## **07-09 케라스의 함수형 API(Keras Functional API)**

앞서 구현한 선형, 로지스틱, 소프트맥스 회귀 모델들과 케라스 훑어보기 실습에서 배운 케라스의 모델 설계 방식은 Sequential API을 사용한 것입니다. 그런데 Sequential API는 여러층을 공유하거나 다양한 종류의 입력과 출력을 사용하는 등의 복잡한 모델을 만드는 일에는 한계가 있습니다. 이번에는 더욱 복잡한 모델을 생성할 수 있는 방식인 Functional API(함수형 API)에 대해서 알아봅니다.  
  
Functional API에 대한 자세한 소개는 케라스 공식 문서에서도 확인할 수 있습니다.  
  
링크 : https://keras.io/getting-started/functional-api-guide/


### **1. Sequential API로 만든 모델**
두 가지 API의 차이를 이해하기 위해서 앞서 배운 Sequential API를 사용하여 기본적인 모델을 만들어봅시다.


In [ ]:
# !pip show tensorflow

In [1]:
!python --version

Python 3.10.0


In [2]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(3, input_dim=4, activation='softmax'))
model.summary()

c:\Users\dlwlg\anaconda3\envs\py310_yolo\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 3)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15 (60.00 B)

 Trainable params: 15 (60.00 B)

 Non-trainable params: 0 (0.00 B)



위와 같은 방식은 직관적이고 편리하지만 단순히 층을 쌓는 것만으로는 구현할 수 없는 복잡한 신경망을 구현할 수 없습니다. 따라서 초심자에게 적합한 API이지만, 전문가가 되기 위해서는 결과적으로 Functional API를 학습해야 합니다.



### **2. Functional API로 만든 모델**
Functional API는 각 층을 일종의 함수(function)로서 정의합니다. 그리고 각 함수를 조합하기 위한 연산자들을 제공하는데, 이를 이용하여 신경망을 설계합니다. Functional API로 FFNN, RNN 등 다양한 모델을 만들면서 기존의 sequential API와의 차이를 이해해봅시다.

* Functional API는 입력의 크기(shape)를 명시한 입력층(Input layer)을 모델의 앞단에 정의해주어야 합니다.  
  
#### **1) 전결합 피드 포워드 신경망(Fully-connected FFNN)**
Sequential API와는 다르게 functional API에서는 입력 데이터의 크기(shape)를 인자로 입력층을 정의해주어야 합니다. 피드 포워드 신경망(Fully-connected FFNN)을 만든다고 가정해보겠습니다.


In [9]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [10]:
inputs = Input(shape=(10,))
inputs

<KerasTensor shape=(None, 10), dtype=float32, sparse=False, ragged=False, name=keras_tensor_13>

위의 코드는 10개의 입력을 받는 입력층을 보여줍니다. 위의 코드에 은닉층과 출력층을 추가해봅시다.

In [12]:
inputs = Input(shape=(10,))
hidden1 = Dense(64, activation='relu')(inputs)  # <- 새로 추가
hidden2 = Dense(64, activation='relu')(hidden1) # <- 새로 추가
output = Dense(1, activation='sigmoid')(hidden2) # <- 새로 추가

위의 코드를 하나의 모델로 구성해보겠습니다. 이는 Model에 입력 텐서와 출력 텐서를 정의하여 완성됩니다.

In [13]:
inputs = Input(shape=(10,))
hidden1 = Dense(64, activation='relu')(inputs)
hidden2 = Dense(64, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=inputs, outputs=output) # <- 새로 추가

In [14]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

지금까지의 내용을 정리하면 다음과 같습니다.

* Input() 함수에 입력의 크기를 정의합니다.
* 이전층을 다음층 함수의 입력으로 사용하고, 변수에 할당합니다.
* Model() 함수에 입력과 출력을 정의합니다.  
  
이를 model로 저장하면 sequential API를 사용할 때와 마찬가지로 model.compile, model.fit 등을 사용 가능합니다.


In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(data, labels)

이번에는 변수명을 달리해서 FFNN을 만들어보겠습니다. 이번에는 은닉층과 출력층의 변수를 전부 x로 통일하였습니다.


In [20]:
inputs = Input(shape=(10,))
x = Dense(8, activation="relu")(inputs)
x = Dense(4, activation="relu")(x)
x = Dense(1, activation="linear")(x)
model = Model(inputs, x)

이번에는 위에서 배운 내용을 바탕으로 선형 회귀와 로지스틱 회귀를 Functional API로 구현해봅시다.


#### **2) 선형 회귀(Linear Regression)**
앞서 ( https://wikidocs.net/111472 ) Sequential API로 구현했던 선형 회귀를 Functional API로 구현해봅시다.


In [3]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import numpy as np
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9]) # 공부하는 시간
y = np.array([11, 22, 33, 44, 53, 66, 77, 87, 95]) # 각 공부하는 시간에 맵핑되는 성적

inputs = Input(shape=(1,))
hidden = Dense(4, activation='relu')(inputs)
output = Dense(1, activation='linear')(hidden)
linear_model = Model(inputs, output)

sgd = optimizers.SGD(learning_rate=0.01)

linear_model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
linear_model.fit(X, y, epochs=300)


Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - loss: 3763.1379 - mse: 3763.1379
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 899.9901 - mse: 899.9901
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 512221.5000 - mse: 512221.5000
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 5039.7739 - mse: 5039.7739
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 4870.3726 - mse: 4870.3726
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 4707.6782 - mse: 4707.6782
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 4551.4277 - mse: 4551.4277
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 4401.3647 - mse: 4401.3647
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 4257.2441 - mse: 4257.2441
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 4118.8301 - mse: 4118.8301
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 3985.8975 - mse: 3985.8975
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 3

그 외에 다양한 다른 예제들을 구현해봅시다.

#### **3) 로지스틱 회귀(Logistic Regression)**


In [8]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(3,))
output = Dense(1, activation='sigmoid')(inputs)
logistic_model = Model(inputs, output)
logistic_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

#### **4) 다중 입력을 받는 모델(model that accepts multiple inputs)**
functional API를 사용하면 아래와 같이 다중 입력과 다중 출력을 가지는 모델도 만들 수 있습니다.


```python
    # 최종 완성된 다중 입력, 다중 출력 모델의 예
    model = Model(inputs=[a1, a2], outputs=[b1, b2, b3])
```

이번에는 다중 입력을 받는 모델을 입력층부터 출력층까지 설계해보겠습니다.

In [1]:
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

# 두 개의 입력층을 정의
inputA = Input(shape=(64,))
inputB = Input(shape=(128,))

# 첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
x = Dense(16, activation="relu")(inputA)
x = Dense(8, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)

# 두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

# 두개의 인공 신경망의 출력을 연결(concatenate)
result = concatenate([x.output, y.output])

z = Dense(2, activation="relu")(result)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │      1,040 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 8)         │        136 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 8)         │        264 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16)        │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 2)         │         34 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │          3 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,813 (46.14 KB)

 Trainable params: 11,813 (46.14 KB)

 Non-trainable params: 0 (0.00 B)


위 모델은 두 개의 입력층으로부터 분기되어 진행된 후 마지막에는 하나의 출력을 예측하는 모델입니다.

#### **5) RNN(Recurrence Neural Network) 은닉층 사용하기**
이번에는 RNN 은닉층을 가지는 모델을 설계해봅시다. 여기서는 하나의 특성(feature)에 50개의 시점(time-step)을 입력으로 받는 모델을 설계해보겠습니다. RNN에 대한 구체적인 내용은 다음 챕터인 RNN 챕터에서 배웁니다.


In [9]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

inputs = Input(shape=(50, 1))
lstm_layer = LSTM(10)(inputs)
x = Dense(10, activation='relu')(lstm_layer)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 50, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 601 (2.35 KB)

 Trainable params: 601 (2.35 KB)

 Non-trainable params: 0 (0.00 B)

다수의 입력과 다수의 출력을 가지는 좀 더 다양한 예제는 앞서 소개한 케라스 공식 문서에서 확인할 수 있습니다.

#### **6) 다르게 보이지만 동일한 표기**
케라스의 Functional API가 익숙하지 않은 상태에서 Functional API를 사용한 코드를 보다가 혼동할 수 있는 점이 한 가지 있습니다. 바로 동일한 의미를 가지지만, 하나의 줄로 표현할 수 있는 코드를 두 개의 줄로 표현한 경우입니다.


In [ ]:
# result = Dense(128)(input)

위 코드는 아래와 같이 두 개의 줄로 표현할 수 있습니다.

In [ ]:
# dense = Dense(128)
# result = dense(input)

마지막 편집일시 : 2022년 11월 14일 2:50 오후